## Improvements/to-do

- slightly change dimensions to account for 0.5um over-etch during fabrication
- ANSYS simulation hfss renderer
- try write RF simulation module for COMSOL

In [1]:
# set Jupyter autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# import qiskit
from qiskit_metal import designs
from qiskit_metal import MetalGUI, Dict

# import components needed for CPW meander and launchpads
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

# for simulation
import pyEPR as epr
import numpy as np

# chip design

https://qiskit.org/documentation/metal/circuit-examples/full-design-flow-examples/Example-full-chip-design.html

In [3]:
%metal_heading Chip setup

In [4]:
# gap and width dimensions
width = '10um'
gap = '4.667um'
substrate_thickness = '-0.53mm'

# chip dimensions
chip_x = '7mm'
chip_y = '4mm'

In [5]:
# Each time you create a new quantum circuit design you start by instantiating a QDesign class.

# The design class `DesignPlanar` is best for 2D circuit designs.
design = designs.DesignPlanar(metadata={}, overwrite_enabled=True, enable_renderers=True)
design.delete_all_components()

In [6]:
# set chip dimension and material
design.chips.main.size.size_x = chip_x
design.chips.main.size.size_y = chip_y
design.chips.main.size.size_z = substrate_thickness
design.chips.main.material = ['sapphire']

# enact
design.chips.main

#create gui
gui = MetalGUI(design)

In [7]:
# setup gds export
#full_chip_gds = design.renderers.gds

# show options
#full_chip_gds.options

# set path to 'resources' folder
#full_chip_gds.options['path_filename'] = '/Users/uqzdegna/Qiskit/qiskit-metal/tutorials/resources/Fake_Junctions.GDS'

In [8]:
%metal_heading Launchpads

In [9]:
# set launchpad options
lp_L_options = Dict(
    pos_x = '-2500um',
    pos_y = '0um',
    pad_width = '400um',
    pad_height = '400um',
    pad_gap = '186.66um',
    trace_width=width,
    trace_gap=gap
)

lp_R_options = Dict(
    pos_x = '2500um',
    pos_y = '0um',
    orientation = '-180',
    pad_width = '400um',
    pad_height = '400um',
    pad_gap = '186.66um',
    trace_width=width,
    trace_gap=gap
)

In [10]:
# place launchpads
lp_L = LaunchpadWirebond(
    design, 
    'lpLeft', 
    options=lp_L_options
    )

lp_R = LaunchpadWirebond(
    design, 
    'lpRight', 
    options=lp_R_options
    )

# show pin names
print(f'Left pin: {lp_L.pin_names}\nRight pin: {lp_R.pin_names}')

Left pin: {'tie'}
Right pin: {'tie'}


In [11]:
%metal_heading Meanders

In [12]:
from qiskit_metal.analyses import cpw_calculations

w = 10*10**-6       # width
g = 4.667*10**-6    # gap
st = 530*10**-6     # substrate thickness
ft = 200*10**-9     # film thickness

# can use length calculator to calculate length of resonator for desired resonance frequency
four_ghz = cpw_calculations.guided_wavelength(
    freq=5*10**9,  
    line_width=w, 
    line_gap=g, 
    substrate_thickness=st, 
    film_thickness=ft
    )

five_ghz = cpw_calculations.guided_wavelength(
    freq=5.5*10**9,  
    line_width=w, 
    line_gap=g, 
    substrate_thickness=st, 
    film_thickness=ft
    )

six_ghz = cpw_calculations.guided_wavelength(
    freq=6*10**9,  
    line_width=w, 
    line_gap=g, 
    substrate_thickness=st, 
    film_thickness=ft
    )

seven_ghz = cpw_calculations.guided_wavelength(
    freq=6.5*10**9,  
    line_width=w, 
    line_gap=g, 
    substrate_thickness=st, 
    film_thickness=ft
    )

eight_ghz = cpw_calculations.guided_wavelength(
    freq=7*10**9,  
    line_width=w, 
    line_gap=g, 
    substrate_thickness=st, 
    film_thickness=ft
    )

print('-------- Wavelength calculations --------')
print('-----------------------------------------')
print(f'5.0 GHz (quarter-wavelength): {four_ghz[0]/4*10**3:4f} mm')
print(f'5.5 GHz (quarter-wavelength): {five_ghz[0]/4*10**3:4f} mm')
print(f'6.0 GHz (quarter-wavelength): {six_ghz[0]/4*10**3:4f} mm')
print(f'6.5 GHz (quarter-wavelength): {seven_ghz[0]/4*10**3:4f} mm')
print(f'7.0 GHz (quarter-wavelength): {eight_ghz[0]/4*10**3:4f} mm')
print('-----------------------------------------')

# can also use kappa_calculation to calculate crosstalk

-------- Wavelength calculations --------
-----------------------------------------
5.0 GHz (quarter-wavelength): 6.101539 mm
5.5 GHz (quarter-wavelength): 5.546854 mm
6.0 GHz (quarter-wavelength): 5.084616 mm
6.5 GHz (quarter-wavelength): 4.693492 mm
7.0 GHz (quarter-wavelength): 4.358242 mm
-----------------------------------------


In [13]:
# OPTIONS
# set start, end, resonator options that can be used for all resonators

tl_y = '1000um' # transmission line y value
res_y = '950um'

# START
start_ops = Dict(
    pos_y=res_y,
    width=width,
    gap=gap,
    termination_gap='10um',
    orientation='180'
) 

# END
end_ops = Dict(
    width=width,
    gap=gap,
    termination_gap='0um',
    orientation='-90'
)

# MEANDER
res_ops = Dict(
    trace_width='10um',
    trace_gap='4.667um',
    fillet='99um',
    start_straight='1um',
    meander = Dict(
        spacing='200um'
    )
)

In [14]:
# resonator 1
r1_start = OpenToGround(
    design, 
    'r1_start', 
    options=Dict(
        pos_x='-1800um', 
        **start_ops
    )
)

r1_end = OpenToGround(
    design, 
    'r1_end', 
    options=Dict(
        pos_x='-1800um',
        pos_y='-1000um', 
        **end_ops
    )
)

r1 = RouteMeander(
    design, 
    'r1', 
    options=Dict(
        # hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component='r1_start',
                pin='open'
            ),
            end_pin=Dict(
                component='r1_end',
                pin='open'
            )
        ),
        total_length='6.101463mm',
        **res_ops
    )
)

# print pin names
print(f'Resonator 1 pin names: {r1.pin_names}')

Resonator 1 pin names: {'end', 'start'}


In [15]:
# resonator 2

r2_start = OpenToGround(
    design, 
    'r2_start', 
    options=Dict(
        pos_x='-900um', 
        **start_ops
    )
)

r2_end = OpenToGround(
    design, 
    'r2_end', 
    options=Dict(
        pos_x='-900um',
        pos_y='-1000um',  
        **end_ops
    )
)

r2 = RouteMeander(
    design, 
    'r2', 
    options=Dict(
        # hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component='r2_start',
                pin='open'
            ),
            end_pin=Dict(
                component='r2_end',
                pin='open'
            )
        ),
        total_length='5.546785mm',
        **res_ops
    )
)

# print pin names
print(f'Resonator 2 pin names: {r2.pin_names}')

Resonator 2 pin names: {'end', 'start'}


In [16]:
# resonator 3

r3_start = OpenToGround(
    design, 
    'r3_start', 
    options=Dict(
        pos_x='0um',  
        **start_ops
    )
)

r3_end = OpenToGround(
    design, 
    'r3_end', 
    options=Dict(
        pos_x='0um',
        pos_y='-800um',  
        **end_ops
    )
)

r3 = RouteMeander(
    design, 
    'r3', 
    options=Dict(
        # hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component='r3_start',
                pin='open'
            ),
            end_pin=Dict(
                component='r3_end',
                pin='open'
            )
        ),
        total_length='5.084553mm',
        **res_ops
    )
)

# print pin names
print(f'Resonator 3 pin names: {r3.pin_names}')

Resonator 3 pin names: {'end', 'start'}


In [17]:
# resonator 4

r4_start = OpenToGround(
    design, 
    'r4_start', 
    options=Dict(
        pos_x='900um', 
        **start_ops
    )
)

r4_end = OpenToGround(
    design, 
    'r4_end', 
    options=Dict(
        pos_x='900um',
        pos_y='-600um', 
        **end_ops
    )
)

r4 = RouteMeander(
    design, 
    'r4', 
    options=Dict(
        # hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component='r4_start',
                pin='open'
            ),
            end_pin=Dict(
                component='r4_end',
                pin='open'
            )
        ),
        total_length='4.693433mm',
        **res_ops
    )
)

# print pin names
print(f'Resonator 4 pin names: {r4.pin_names}')

Resonator 4 pin names: {'end', 'start'}


In [18]:
# resonator 5

r5_start = OpenToGround(
    design, 
    'r5_start', 
    options=Dict(
        pos_x='1800um', 
        **start_ops
    )
)

r5_end = OpenToGround(
    design, 
    'r5_end', 
    options=Dict(
        pos_x='1800um',
        pos_y='-600um', 
        **end_ops
    )
)

r5 = RouteMeander(
    design, 
    'r5', 
    options=Dict(
        # hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component='r5_start',
                pin='open'
            ),
            end_pin=Dict(
                component='r5_end',
                pin='open'
            )
        ),
        total_length='4.358188mm',
        **res_ops
    )
)

# print pin names
print(f'Resonator 5 pin names: {r5.pin_names}')

Resonator 5 pin names: {'end', 'start'}


In [19]:
%metal_heading Transmission line

In [20]:
# set transmission line options

straight_length = '70um'

tl_options = Dict(
    # hfss_wire_bonds = True,
    pin_inputs = Dict(
        start_pin = Dict(component='lpLeft', pin='tie'),
        end_pin = Dict(component='lpRight', pin='tie')
    ),
    anchors=Dict(
        anchor_tl=('0um','1000um')
    ),
    lead = Dict(
        start_straight=straight_length,
        end_straight=straight_length
    ),
    fillet = '69um',
    advanced = Dict(avoid_collision='true'),
    trace_width=width,
    trace_gap=gap
)

# draw transmission line
tl = RouteAnchors(
    design, 
    'tl', 
    options=tl_options
)

# print transmission line pin names
print(f'Transmission line pin names (for excitation in ANSYS): {tl.pin_names}')

gui.rebuild()

Transmission line pin names (for excitation in ANSYS): {'end', 'start'}
